In [3]:
# Library
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [4]:
words_data = pd.read_csv('./Data/Preprocess_Data/up20_words_완료_추가제거x.csv')
words = pd.read_csv('./Data/Preprocess_Data/추가할 단어0720.csv')

In [5]:
words

,keywords
0,1d_cnn
1,1d_convolutional_neural_network
2,1dcnn
3,28_ghz
4,2d_human_pose_estimation
...,...
395,yolov5
396,yolov5_algorithm
397,yolov5s
398,yolov7


In [4]:
words_data = words_data[words_data['제거']!='x']
words_data.reset_index(drop=True, inplace=True)
words_data.drop('제거',axis=1, inplace=True)
len(words_data)

9187

In [5]:
words_data = words_data[words_data['추가제거']!='x']

In [12]:
words_data = words_data[['word']]

In [14]:
words_data

,word
0,802.11
1,2d animation
2,2d barcode
3,2d cnn
4,2d image
...,...
9182,zero watermarking
9183,zigbee
9184,zigbee network
9185,zigbee protocol


In [7]:
words.columns = ['word2']

In [8]:
words['word'] = words['word2'].str.replace('_', ' ')

In [10]:
words = words[['word']]

In [11]:
words.head()

,word
0,1d cnn
1,1d convolutional neural network
2,1dcnn
3,28 ghz
4,2d human pose estimation


In [15]:
final_words = pd.concat([words_data, words])

In [17]:
final_words.reset_index(drop=True, inplace=True)
len(final_words)

9153

In [18]:
# 양쪽 공백 제거
final_words['word'] = final_words['word'].str.strip()

In [19]:
# final_words에 공백을 언더바(_)로 대체
final_words['word2'] = 0
for i in range(len(final_words)):
    if len(str(final_words['word'][i]).split(' ')) >1:
        final_words['word2'][i] = final_words['word'][i].replace(' ','_')
    else:
        final_words['word2'][i] = final_words['word'][i]

In [20]:
final_words['length'] = 0
for i in range(len(final_words)):
    final_words['length'][i] = len(str(final_words['word'][i]))

In [21]:
# 결과 확인
final_words.head()

,word,word2,length
0,802.11,802.11,6
1,2d animation,2d_animation,12
2,2d barcode,2d_barcode,10
3,2d cnn,2d_cnn,6
4,2d image,2d_image,8


In [22]:
# length에 따라 정렬 (변환이 제대로 되지 않는 것 방지)
final_words.sort_values(by='length', ascending=False, inplace=True)
final_words.reset_index(drop=True, inplace=True)

In [23]:
# 중복되는 값 중 첫번째만 남기고 제거
final_words.drop_duplicates(subset='word2', keep='first', inplace=True)

In [24]:
final_words

,word,word2,length
0,bidirectional encoder representations from tra...,bidirectional_encoder_representations_from_tra...,55
1,bidirectional encoder representations from tra...,bidirectional_encoder_representations_from_tra...,54
2,deep convolutional generative adversarial network,deep_convolutional_generative_adversarial_network,49
3,contrast limited adaptive histogram equalization,contrast_limited_adaptive_histogram_equalization,48
4,covariance matrix adaptation evolution strategy,covariance_matrix_adaptation_evolution_strategy,47
...,...,...,...
9148,ar,ar,2
9149,4k,4k,2
9150,3d,3d,2
9151,6g,6g,2


In [25]:
final_words.to_csv('Data/Preprocess_Data/final_words.csv', encoding='utf-8-sig', index=False)